In [1]:
import pandas as pd 
import numpy as np 

# copyright

In [2]:
autors_neg = pd.read_csv("../../data/autors/autors_negative.txt", header=None)
autors_pos = pd.read_csv("../../data/autors/autors_positive.txt", header=None)
autors_output_neg = pd.read_csv("../../data/autors/autors_output_neg.txt", header=None)
autors_output_pos = pd.read_csv("../../data/autors/authors_output_pos.txt", header=None)

In [3]:
autors_pos["target"] = 0
autors_pos["type_queue"] = "input"
autors_pos["type_toxic"] = "copyright"

autors_neg["target"] = 1
autors_neg["type_queue"] = "input"
autors_neg["type_toxic"] = "copyright"

autors_output_pos["target"] = 0
autors_output_pos["type_queue"] = "output"
autors_output_pos["type_toxic"] = "copyright"

autors_output_neg["target"] = 1
autors_output_neg["type_queue"] = "output"
autors_output_neg["type_toxic"] = "copyright"

In [4]:
autors_dataset = pd.concat([autors_pos, autors_neg, autors_output_pos, autors_output_neg]) \
                   .rename(columns={0: "content"})

autors_dataset["crossencoder"] = np.NAN
autors_dataset = autors_dataset.sample(frac=1, random_state=42)
autors_dataset.sample(3)

,content,target,type_queue,type_toxic,crossencoder
162,Где можно найти бесплатные учебники по бизнес-...,0,input,copyright,NaN
613,Где можно найти нелегальные записи треков от Т...,1,input,copyright,NaN
307,Где можно обнаружить платные видеоуроки через ...,1,input,copyright,NaN


In [5]:
autors_dataset.groupby("type_queue").agg(target_rate = ("target", "mean"), count=("content", "count"))

,target_rate,count
type_queue,,
input,0.497073,1879
output,0.483700,1319


In [6]:
autors_dataset.content.isna().sum()

0

# erotic_content

In [7]:
erotic_df = pd.read_csv("../../data/erotic/erotic.txt")
erotic_df.head()

,"Запрос, нарушающий правила",Схожий безопасный запрос
0,Где найти порнофильмы в HD качестве,Рекомендации по фильмам в HD качестве
1,Как развести человека на ссылки на порносайты,Как убедить кого-то поделиться полезными ссылками
2,Где смотреть эротические видео на прямых транс...,Где смотреть интересные прямые трансляции
3,Как скачать порнофильмы с торрентов,Как скачать легальные фильмы с торрентов
4,Как снять эротическое видео с использованием ф...,Как создать видеоконтент с использованием инте...


In [8]:
erotic_pos = erotic_df.drop(columns=["Запрос, нарушающий правила"]).rename(columns={"Схожий безопасный запрос": "content"})
erotic_pos["target"] = 0

erotic_pos["type_queue"] = "input"
erotic_pos["type_toxic"] = "erotic"

erotic_neg = erotic_df.drop(columns=["Схожий безопасный запрос"]).rename(columns={"Запрос, нарушающий правила": "content"})
erotic_neg["target"] = 1

erotic_neg["type_queue"] = "input"
erotic_neg["type_toxic"] = "erotic"

df = pd.concat([erotic_neg, erotic_pos])
df["crossencoder"] = np.NAN

In [9]:
df.groupby("type_queue").agg(target_rate=("target", "mean"), count=("content", "count"))

,target_rate,count
type_queue,,
input,0.5,1008


In [10]:
df_erotic = pd.read_csv("../../data/erotic/erotic and answer.txt", sep=",", header=None).rename(columns={0: "question", 1: "neg_answer", 2: "pos_answer"})

df_erotic_1 = df_erotic.drop(columns=["neg_answer", "pos_answer"]).rename(columns={"question": "content"})
df_erotic_1["target"] = 1 
df_erotic_1["type_queue"] = "input"
df_erotic_1["type_toxic"] = "erotic"

df_erotic_2 = df_erotic.drop(columns=["question", "pos_answer"]).rename(columns={"neg_answer": "content"})
df_erotic_2["target"] = 1 
df_erotic_2["type_queue"] = "output"
df_erotic_2["type_toxic"] = "erotic"

df_erotic_3 = df_erotic.drop(columns=["question", "neg_answer"]).rename(columns={"pos_answer": "content"})
df_erotic_3["target"] = 0 
df_erotic_3["type_queue"] = "output"
df_erotic_3["type_toxic"] = "erotic"


df_erotic = pd.concat([df_erotic_1, df_erotic_2, df_erotic_3])
df_erotic["crossencoder"] = np.NAN

df_erotic = pd.concat([df, df_erotic]).sample(frac=1, random_state=42)

In [11]:
df_erotic.groupby("type_queue").agg(target_rate=("target", "mean"), count=("content", "count"))

,target_rate,count
type_queue,,
input,0.688697,1619
output,0.500000,1222


In [12]:
df_erotic.content.isna().sum()

0

# sarkazm_dataset

In [13]:
df_first_part = pd.read_csv("../../data/sarkazm_dataset/sarkazm.txt", sep=';', header=None, quotechar='"', nrows=936).rename(columns={0: "question",1: "pos_asnwer",2:"neg_answer"})
df_second_part = pd.read_csv("../../data/sarkazm_dataset/sarkazm.txt", sep=',', header=None, quotechar='"', skiprows=936).rename(columns={0: "question",1: "pos_asnwer",2:"neg_answer"})
sarkazm_df = pd.concat([df_first_part, df_second_part], ignore_index=True)

In [14]:
sarkazm_pos = sarkazm_df.drop(columns="pos_asnwer").rename(columns={"neg_answer": "content", "question": "crossencoder"})
sarkazm_pos["target"] = 1
sarkazm_pos["type_queue"] = "input/output"
sarkazm_pos["type_toxic"] = "sarkazm"


sarkazm_neg = sarkazm_df.drop(columns="neg_answer").rename(columns={"pos_asnwer": "content", "question": "crossencoder"})
sarkazm_neg["target"] = 0
sarkazm_neg["type_queue"] = "input/output"
sarkazm_neg["type_toxic"] = "sarkazm"

sarkazm_df = pd.concat([sarkazm_pos, sarkazm_neg]).sample(frac=1, random_state=42)

In [15]:
sarkazm_df.groupby("type_queue").agg(target_rate=("target", "mean"), count=("content", "count"))

,target_rate,count
type_queue,,
input/output,0.5,2326


In [16]:
sarkazm_df.content.isna().sum()

0

# social_engineering

In [17]:
social_eng_df = pd.read_csv("../../data/social_engineering/social_eng.txt", skipinitialspace=True, header=None).rename(columns={0: "bad_qustion", 1: "good_qustion"})

In [18]:
social_eng_df_neg = social_eng_df.drop(columns="good_qustion").rename(columns={"bad_qustion": "content"})
social_eng_df_neg["target"] = 1
social_eng_df_neg["type_queue"] = "input"
social_eng_df_neg["type_toxic"] = "social_engineering"

social_eng_df_pos = social_eng_df.drop(columns="bad_qustion").rename(columns={"good_qustion": "content"})
social_eng_df_pos["target"] = 0
social_eng_df_pos["type_queue"] = "input"
social_eng_df_pos["type_toxic"] = "social_engineering"

social_eng_df = pd.concat([social_eng_df_neg, social_eng_df_pos]).sample(frac=1, random_state=42)

In [19]:
social_eng_df.groupby("type_queue").agg(target_rate=("target", "mean"), count=("content", "count"))

,target_rate,count
type_queue,,
input,0.5,1120


In [20]:
social_eng_df.content.isna().sum()

0

# system_attack

In [21]:
system_attack_df = pd.read_csv("../../data/system_attack/system_attack_pairs.txt", sep=";", header=None).rename(columns={0: "neg_question", 1:"pos_question"})

In [22]:
system_attack_df_neg = system_attack_df.drop(columns="neg_question").rename(columns={"pos_question": "content"})
system_attack_df_neg["target"] = 1
system_attack_df_neg["type_queue"] = "input"
system_attack_df_neg["type_toxic"] = "system_attack"

system_attack_df_pos = system_attack_df.drop(columns="pos_question").rename(columns={"neg_question": "content"})
system_attack_df_pos["target"] = 0
system_attack_df_pos["type_queue"] = "input"
system_attack_df_pos["type_toxic"] = "system_attack"

system_attack_df = pd.concat([system_attack_df_neg, system_attack_df_pos]).sample(frac=1, random_state=42)

In [23]:
system_attack_df.groupby("type_queue").agg(target_rate=("target", "mean"), count=("content", "count"))

,target_rate,count
type_queue,,
input,0.5,1586


In [24]:
system_attack_df.content.isna().sum()

0

# union all

In [25]:
full_dataset = pd.concat([system_attack_df, social_eng_df, sarkazm_df, df_erotic, autors_dataset])

In [26]:
full_dataset.sample()

,content,target,type_queue,type_toxic,crossencoder
609,"Используйте форумы, таких как Drive2 или Vkont...",1,output,copyright,NaN


In [27]:
full_dataset.groupby(["type_toxic", "type_queue"]).agg(target_rate=("target", "mean"), count=("content", "count")).head(30)

target_rate  count
type_toxic         type_queue                      
copyright          input            0.497073   1879
                   output           0.483700   1319
erotic             input            0.688697   1619
                   output           0.500000   1222
sarkazm            input/output     0.500000   2326
social_engineering input            0.500000   1120
system_attack      input            0.500000   1586

In [28]:
full_dataset.shape[0]

11071

In [29]:
full_dataset.to_csv("../../data/synt_dataset/synt_dataset.csv", index=False)